In [1]:
import os
import torch
import numpy as np
import torchvision
from PIL import Image
import matplotlib.pyplot as plt

import torchvision.transforms as T
import torch.nn.functional as F

from ig_pkg.datasets import get_datasets

from ig_pkg.models.generator import get_model
from ig_pkg.models.classifier import get_classifier
from ig_pkg.models.pretrained_models import get_pretrained_model

from ig_pkg.inputattribs.ig import ig
from ig_pkg.inputattribs.baseline_generator import get_baseline_generator

from ig_pkg.misc import process_heatmap, normalize_tensor, convert_to_img, label_to_class, tran, na_imshow

IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD  = [0.229, 0.224, 0.225]


In [24]:
# data_path="/root/data/ILSVRC2012_val/"

# _, valid_datasets = get_datasets("imagenet1k", data_path)
# _, valid_datasets2 = get_datasets("imagenet1k", data_path, transform=T.Compose([T.Resize(256),T.CenterCrop(256),]) )

device="cuda:0"
# model = get_pretrained_model("vgg16").to(device)

# train_dataset, valid_dataset = get_datasets(name= 'celebAHQ_identity', data_path = '/root/data/CelebA_HQ_facial_identity_dataset')
# class_names = train_dataset.classes
# np.save('/root/data/celebAHQ_identity_class_names.npy', class_names)
# class_names = np.load('/root/data/celebAHQ_identity_class_names.npy')

# train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
# test_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=16, shuffle=True, num_workers=2)

# image_path = '/root/data/CelebA_HQ_facial_identity_dataset/train'

In [ ]:

names = {
    'zero',
    'min',
    'max',
    'scalar',
    'encoder',
    'optimizer',
    'hybrid',
    'gaussian_blur',
    'gaussian',
    'uniform',
    'maximumdistance' 
}

index = 0 
x,y = valid_datasets[index]
import numpy as np
import matplotlib.pyplot as plt 
fig, axes = plt.subplots(2, (len(names)//2)+1, figsize=(2*len(names)//2, 2*2))
axes_flat = axes.flat 
my_cmap=plt.cm.seismic(np.arange(plt.cm.seismic.N))

ax = next(axes_flat)
ax.imshow(valid_datasets2[index][0])

for name in names:
    try:
        b_generator =  get_baseline_generator(name)
        baseline = b_generator(x=x,y=y)

    except Exception as e: 
        print("-----")
        print("fail:", name)
        print(e)
        continue
        
    attr = ig(model, x, y, baseline, device=device)
    attr, kwargs  = process_heatmap(attr.cpu(), my_cmap)
    ax = next(axes_flat)
    ax.imshow(attr.numpy(), **kwargs)
    ax.set_title(name)
    ax.axis("off")
plt.tight_layout()